In [1]:
library(tidyverse)
library(RPostgreSQL)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.2     ✔ dplyr   0.7.4
✔ tidyr   0.8.0     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: DBI


In [2]:
source("utils.R")

In [3]:
sql_dir <- file.path("..", "sql")

In [4]:
drv <- dbDriver("PostgreSQL")
con <- dbConnect(drv, dbname = "mimic")
dbSendQuery(con, "set search_path=echo,public,mimiciii;")

<PostgreSQLResult>

In [5]:
cat("-- Generate view population --\n")

sql <- file_to_sql_view(file.path(sql_dir, "population.sql"),
                        "population", "view")
dbSendQuery(con, sql)

-- Generate view population --


<PostgreSQLResult>

In [6]:
c("cohort", "basics", "icd9", "drugs", "lab_tests", "vital_signs") %>%
walk(function(x) {
    cat(sprintf("-- Generate materialized view %s --\n", x))
    
    x %>%
    file_to_sql_view(file.path(sql_dir, paste0(., ".sql")), .) %>%
    dbSendQuery(con, .)
})

-- Generate materialized view cohort --
-- Generate materialized view basics --
-- Generate materialized view icd9 --
-- Generate materialized view drugs --
-- Generate materialized view lab_tests --
-- Generate materialized view vital_signs --


In [7]:
vs <- "select distinct(label) from vital_signs" %>%
    dbGetQuery(con, .) %>%
    pull(label)
vs

[1] "heart_rate" "cvp"        "map"        "temp"

In [8]:
sql_template <- "
with summary as (
    select distinct icustay_id, label
    , first_value(valuenum) over (partition by icustay_id, label order by charttime) as fst_val
    , first_value(valuenum) over (partition by icustay_id, label order by valuenum) as min_val
    , first_value(valuenum) over (partition by icustay_id, label order by valuenum desc) as max_val
    from vital_signs
)

select icustay_id
, %s
from summary
group by icustay_id
"

In [9]:
sql <- c("max(case when label = '%1$s' then 1 else 0 end) as vs_%1$s_flag",
         "max(case when label = '%1$s' then fst_val else null end) as vs_%1$s_first",
         "max(case when label = '%1$s' then min_val else null end) as vs_%1$s_min",
         "max(case when label = '%1$s' then max_val else null end) as vs_%1$s_max") %>%
paste(collapse = "\n, ") %>%
sprintf(vs) %>%
paste(collapse = "\n, ") %>%
sprintf(sql_template, .)
cat(sql, file = file.path(sql_dir, "vital_signs_unpivot.sql"))

In [10]:
labs <- "select distinct(label) from lab_tests" %>%
    dbGetQuery(con, .) %>%
    pull(label)
labs

[1] "hemoglobin"        "platelet"          "creatinine_kinase"
 [4] "wbc"               "ph"                "chloride"         
 [7] "sodium"            "bun"               "bicarbonate"      
[10] "bnp"               "pco2"              "creatinine"       
[13] "potassium"         "troponin"          "po2"              
[16] "lactate"

In [11]:
sql_template <- "
with lab_summary as (
    select distinct hadm_id, label
    , first_value(valuenum) over (partition by hadm_id, label order by charttime) as fst_val
    , first_value(valuenum) over (partition by hadm_id, label order by valuenum) as min_val
    , first_value(valuenum) over (partition by hadm_id, label order by valuenum desc) as max_val
    , first_value(abnormal) over (partition by hadm_id, label order by abnormal desc) as abnormal
    from lab_tests
)

select hadm_id
, %s
from lab_summary
group by hadm_id
"

In [12]:
sql <- c("max(case when label = '%1$s' then 1 else 0 end) as lab_%1$s_flag",
         "max(case when label = '%1$s' then fst_val else null end) as lab_%1$s_first",
         "max(case when label = '%1$s' then min_val else null end) as lab_%1$s_min",
         "max(case when label = '%1$s' then max_val else null end) as lab_%1$s_max",
         "max(case when label = '%1$s' then abnormal else null end) as lab_%1$s_abnormal") %>%
paste(collapse = "\n, ") %>%
sprintf(labs) %>%
paste(collapse = "\n, ") %>%
sprintf(sql_template, .)
cat(sql, file = file.path(sql_dir, "lab_unpivot.sql"))

In [13]:
c("vital_signs", "lab") %>%
walk(function(x) {
    cat(sprintf("-- Generate materialized view %s_unpivot --\n", x))
    
    x %>%
    { file_to_sql_view(file.path(sql_dir, sprintf("%s_unpivot.sql", .)),
                       sprintf("%s_unpivot", .)) } %>%
    dbSendQuery(con, .)
})

-- Generate materialized view vital_signs_unpivot --
-- Generate materialized view lab_unpivot --


In [14]:
cat("-- Generate materialized view merged_data --\n")

sql <- file_to_sql_view(file.path(sql_dir, "merge_data.sql"),
                        "merged_data")
dbSendQuery(con, sql)

-- Generate materialized view merged_data --


<PostgreSQLResult>

In [15]:
dbDisconnect(con)
dbUnloadDriver(drv)

[1] TRUE

[1] TRUE